In [1]:
import pandas as pd

In [2]:
df=pd.read_json("processed_crypto.json")
df_bitcoin=df[df['asset_name']=='Bitcoin']

In [3]:
df_date_created=df_bitcoin.sort_values('created_utc')
df_date_created['created_utc']=df_date_created['created_utc'].str.slice(0,10)
df_date_created['sma20']=df_date_created['open'].rolling(window=20).mean()
df_date_created['high-low']=df_date_created['sma20'].shift(-1)-df_date_created['sma20']

In [4]:
df_date_created['buy-sell'] = df_date_created['high-low'].apply(lambda x: 0 if x <=0 else 1)

In [5]:
train_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2018')]
train_xx=train_x
train_y=train_x['buy-sell']
train_x=train_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell',])


In [6]:
test_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2019') ]
test_xx=test_x
test_y=test_x['buy-sell']
test_x=test_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell'])


In [7]:
from sklearn.neural_network import MLPClassifier

In [8]:
model=MLPClassifier()

In [10]:
train_x=train_x.fillna(train_x.mean())
test_x=test_x.fillna(test_x.mean())

In [11]:
model.fit(train_x,train_y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [12]:
predict=model.predict(test_x)

In [13]:
from sklearn.metrics import classification_report

In [14]:
print(classification_report(list(test_y),predict))

              precision    recall  f1-score   support

           0       0.52      1.00      0.68       618
           1       0.00      0.00      0.00       581

   micro avg       0.52      0.52      0.52      1199
   macro avg       0.26      0.50      0.34      1199
weighted avg       0.27      0.52      0.35      1199



/Users/mananparasher/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
from sklearn.metrics import accuracy_score

In [16]:
print(accuracy_score(list(test_y),predict))

0.5154295246038365
